This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 14.25 Å
- Li top: frozen top 14.25 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 15-07-2025


In [1]:
structure_name = "LLZO_010_La_order0_off__Li_100_slab_heavy"

In [2]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = (f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_010_La_order0_off__Li_100_slab_heavy
- Initial lattice height: 71.18 Å
- Number of atoms: 600


In [3]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 17.37 Å
Lowest Li slab atom: 38.81 Å
LLZO top z: 34.81 Å
Li penetration into LLZO: -4.00 Å


In [4]:
from pymatgen.core import Structure
import numpy as np

# Load structure
structure = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get z-coordinates and element types
z_coords = np.array([site.z for site in structure.sites])
species = np.array([site.species_string for site in structure.sites])

# LLZO: non-Li atoms (La, Zr, O)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()
llzo_bottom = llzo_z.min()
llzo_thickness = llzo_top - llzo_bottom

# Li slab: Li atoms ABOVE LLZO (i.e. in metallic Li layer)
li_slab_z = np.array([
    site.z for site in structure.sites
    if site.species_string == "Li" and site.z > llzo_top
])
li_thickness = li_slab_z.ptp() if len(li_slab_z) > 0 else 0
li_bottom = li_slab_z.min() if len(li_slab_z) > 0 else None

# Penetration check
penetration = llzo_top - li_bottom if li_bottom is not None else 0

# Report
print(f"LLZO slab thickness: {llzo_thickness:.2f} Å")
print(f"Li slab thickness:   {li_thickness:.2f} Å")
print(f"LLZO top z:          {llzo_top:.2f} Å")
print(f"Lowest Li atom z:    {li_bottom:.2f} Å" if li_bottom else "No Li slab atoms found")
print(f"Li penetration into LLZO: {penetration:.2f} Å")


LLZO slab thickness: 19.81 Å
Li slab thickness:   17.37 Å
LLZO top z:          34.81 Å
Lowest Li atom z:    38.81 Å
Li penetration into LLZO: -4.00 Å


In [5]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 0.75 * (llzo_thickness)  # in Å
freeze_thickness_li = 0.75 * (llzo_thickness)  # in Å
llzo_z_threshold = z_min + freeze_thickness_llzo
li_z_threshold = z_max - freeze_thickness_li

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)

Freezing 427 atoms out of 600
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_159204/3870098642.py:27: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [6]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 14:42:58    -2353.464460        0.914871
SciPyFminCG:    1 14:43:00    -2358.013058        4.578363
SciPyFminCG:    2 14:43:00    -2370.043802        3.882914
SciPyFminCG:    3 14:43:01    -2372.525597        3.898826
SciPyFminCG:    4 14:43:02    -2382.282114        2.030108
SciPyFminCG:    5 14:43:03    -2393.836641        3.475124
SciPyFminCG:    6 14:43:04    -2398.233318        2.737126
SciPyFminCG:    7 14:43:05    -2403.590870        2.929437
SciPyFminCG:    8 14:43:06    -2404.944992        1.658984
SciPyFminCG:    9 14:43:07    -2405.706024        1.120059
SciPyFminCG:   10 14:43:07    -2406.465054        2.905973
SciPyFminCG:   11 14:43:08    -2408.546162        3.823580
SciPyFminCG:   12 14:43:09    -2414.098549        4.260254
SciPyFminCG:   13 14:43:10    -2422.554016        7.022680
SciPyFminCG:   14 14:43:11    -2430.085373        6.393492
SciPyFminCG:   15 14:43:12    -2434.701061        5.146649

In [7]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_159204/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 14:44:40    -2490.692139        0.147193
FIRE:    1 14:44:41    -2490.695858        0.143055
FIRE:    2 14:44:41    -2490.701580        0.135509
FIRE:    3 14:44:41    -2490.708160        0.124559
FIRE:    4 14:44:42    -2490.715027        0.110600
FIRE:    5 14:44:42    -2490.721035        0.095497
FIRE:    6 14:44:43    -2490.726471        0.081072
FIRE:    7 14:44:43    -2490.732193        0.077537
FIRE:    8 14:44:43    -2490.739346        0.078103
FIRE:    9 14:44:44    -2490.747929        0.071461
FIRE:   10 14:44:44    -2490.757656        0.072795
FIRE:   11 14:44:45    -2490.767956        0.073980
FIRE:   12 14:44:45    -2490.779972        0.079352
FIRE:   13 14:44:46    -2490.795135        0.083772
FIRE:   14 14:44:46    -2490.813446        0.074672
FIRE:   15 14:44:46    -2490.833759        0.072100
FIRE:   16 14:44:47    -2490.857792        0.085359
FIRE:   17 14:44:47    -2490.885544        0.086418
FIRE:   18 14:

In [8]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"relaxed_{structure_name}.cif", final_structure_ase)
write(f"relaxed_{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [11]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 21.64 Å
Lowest Li slab atom: 34.54 Å
LLZO top z: 34.42 Å
Li penetration into LLZO: -0.12 Å


In [12]:
import os
from chgnet.model import StructOptimizer
from pymatgen.core import Structure
from chgnet.model.dynamics import CHGNetCalculator

structure_path = f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif"
structure = Structure.from_file(structure_path)

# Output path
output_dir = f"/home/mehuldarak/summer/relax_final"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"cellrelaxed_{structure_name}.cif")

# Run CHGNet relaxation
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result = opt1.relax(
    structure,
    fmax=0.15,           # You can adjust depending on accuracy/speed tradeoff
    steps=400,
    relax_cell=True,
    verbose=True
)


             Step     Time          Energy          fmax
SciPyFminCG:    0 14:47:28    -2492.811584        1.252102
SciPyFminCG:    1 14:47:29    -2494.397736        0.796290
SciPyFminCG:    2 14:47:30    -2496.472263        1.118920
SciPyFminCG:    3 14:47:31    -2497.276497        0.785982
SciPyFminCG:    4 14:47:33    -2498.539066        1.010930
SciPyFminCG:    5 14:47:34    -2499.690914        0.763360
SciPyFminCG:    6 14:47:35    -2500.958061        1.011148
SciPyFminCG:    7 14:47:36    -2501.719093        0.746864
SciPyFminCG:    8 14:47:37    -2503.020573        1.301692
SciPyFminCG:    9 14:47:38    -2504.315472        0.735513
SciPyFminCG:   10 14:47:38    -2505.318260        0.719968
SciPyFminCG:   11 14:47:39    -2505.633831        0.719747
SciPyFminCG:   12 14:47:40    -2506.054974        1.596963
SciPyFminCG:   13 14:47:41    -2506.561947        1.366285
SciPyFminCG:   14 14:47:42    -2507.410812        1.276265
SciPyFminCG:   15 14:47:43    -2508.272552        0.683765

In [13]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=True, verbose=True)

/tmp/ipykernel_159204/1077038773.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 14:50:16    -2594.612503        0.146868
FIRE:    1 14:50:17    -2594.613934        0.146857
FIRE:    2 14:50:17    -2594.616222        0.146838
FIRE:    3 14:50:17    -2594.619370        0.146808
FIRE:    4 14:50:18    -2594.622517        0.146772
FIRE:    5 14:50:18    -2594.625950        0.146732
FIRE:    6 14:50:18    -2594.629669        0.146683
FIRE:    7 14:50:18    -2594.634247        0.146618
FIRE:    8 14:50:19    -2594.639397        0.146516
FIRE:    9 14:50:19    -2594.645977        0.146375
FIRE:   10 14:50:20    -2594.653416        0.146179
FIRE:   11 14:50:20    -2594.662285        0.145906
FIRE:   12 14:50:20    -2594.673443        0.145533
FIRE:   13 14:50:20    -2594.686317        0.145029
FIRE:   14 14:50:21    -2594.701767        0.144362
FIRE:   15 14:50:21    -2594.719791        0.143501
FIRE:   16 14:50:21    -2594.740963        0.142443
FIRE:   17 14:50:22    -2594.765854        0.141182
FIRE:   18 14:

In [ ]:
# Save relaxed structure
relaxed_structure = result["final_structure"]
relaxed_structure.to(filename=output_path)

print(f"✅ Relaxed structure saved to: {output_path}")

✅ Relaxed structure saved to: /home/mehuldarak/summer/relax_final/cellrelaxed_LLZO_010_La_order0_off__Li_100_slab_heavy.cif


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1724: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


: 